In [1]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 12.5 MB/s eta 0:00:00


In [ ]:
!pip install -q chromadb

# Document Search: Embeddings

In [20]:
import google.generativeai as palm

import textwrap
import numpy as np
import pandas as pd
import chromadb
from chromadb.api.types import Documents, Embeddings

In [3]:
palm.configure(api_key='')

In [21]:
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]

model = models[0]
model

Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representation of a text.',
      input_token_limit=1024,
      output_token_limit=1,
      supported_generation_methods=['embedText'],
      temperature=None,
      top_p=None,
      top_k=None)

In [5]:
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "PaLM API & MakerSuite: An approachable way to explore and prototype with generative AI applications")

# Create an embedding
embedding = palm.generate_embeddings(model=model, text=sample_text)

print(embedding)

{'embedding': [0.012246046, -0.023558903, 0.032459036, 0.06484912, 0.026284628, -0.052756585, 0.0055233696, 0.011005492, -0.03862501, -0.018264746, 0.06678695, -0.015016806, 0.0035746037, -0.009914331, -0.022514464, 0.030050583, -0.078709245, -0.0015311453, -0.02805761, 0.0036338808, -0.076459445, 0.009172192, 0.01225061, -0.016513903, 0.008183921, -0.08033063, 0.028195586, 0.029587045, -0.031249639, -0.019803159, 0.0025109726, 0.018474173, -0.006070546, 0.0042981566, 0.010934953, 0.03646359, -0.027267052, 0.006511828, 0.017602839, 0.015774623, 0.042609964, -0.04978823, 0.021985881, -0.0018003813, 0.015031357, 0.03226512, -0.049656868, 0.0056817518, 0.037447836, -0.082058676, 0.0070665455, -0.009903009, -0.0012817691, -0.009555456, 0.013598595, 0.07107551, -0.10985609, 0.00044024497, -0.027354741, -0.021102894, -0.0077857957, 0.030045867, 0.0065566953, -0.02866328, -0.024084672, 0.027182486, 0.010249044, 0.028895397, -0.048748404, 0.0144549105, 0.035357818, 0.026979432, -0.011489553, -

### Building an embeddings database

In [7]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console. Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

texts = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [8]:
df = pd.DataFrame(texts)
df.columns = ['Text']
df

,Text
0,Operating the Climate Control System Your Goo...
1,Your Googlecar has a large touchscreen display...
2,Shifting Gears Your Googlecar has an automati...


In [9]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(text):
  return palm.generate_embeddings(model=model, text=text)['embedding']

df['Embeddings'] = df['Text'].apply(embed_fn)
df

,Text,Embeddings
0,Operating the Climate Control System Your Goo...,"[-0.015123772, 0.053951535, 0.010618031, 0.046..."
1,Your Googlecar has a large touchscreen display...,"[-0.021975275, 0.062008664, 0.011442106, 0.054..."
2,Shifting Gears Your Googlecar has an automati...,"[-0.017382653, 0.023597008, 0.026251236, 0.038..."


In [10]:
query = "How do you shift gears in the Google car?"

In [12]:
passage = find_best_passage(query, df)
passage

'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.'

## Question and Answering Application

In [13]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [14]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position i

In [15]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

text_model = text_models[0]

In [35]:
temperature = 0.5
answer = palm.generate_text(prompt=prompt,
                            model=text_model,
                            candidate_count=3,
                            temperature=temperature,
                            max_output_tokens=1000)

In [36]:
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

Candidate 0: To shift gears in the Google car, simply move the shift lever to the desired position. Park, Reverse, Neutral, Drive, and Low.

Candidate 1: To shift gears in the Google car, simply move the shift lever to the desired position. The shift lever is located on the center console.

Candidate 2: To shift gears in your Google car, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move.  Reverse: This position is used to back up.  Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal.  Drive: This position is used to drive forward.  Low: This position is used for driving in snow or other slippery conditions.



### Creating the embedding database with ChromaDB

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        # embed the documents somehow
        return embeddings

In [22]:
def embed_function(texts: Documents) -> Embeddings:
  # Embed the documents using any supported method
  return  [palm.generate_embeddings(model=model, text=text)['embedding']
           for text in texts]

In [23]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=embed_function)
  for i,d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [24]:
# Set up the DB
db = create_chroma_db([DOCUMENT1, DOCUMENT2, DOCUMENT3], "googlecardb")

In [25]:
pd.DataFrame(db.peek(1))

,ids,embeddings,metadatas,documents
0,0,"[-0.01512377243489027, 0.05395153537392616, 0....",None,Operating the Climate Control System Your Goo...


In [30]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [31]:
# Perform embedding search
passage = get_relevant_passage("touch screen features", db)
passage

'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'

In [32]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
text_model = text_models[0]
text_model

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='Text Bison',
      description='Model targeted for text generation.',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [33]:
def answer(model, query, db, temperature=0.5):
  passage = get_relevant_passage(query, db)
  prompt = make_prompt(query, passage)
  answer = palm.generate_text(prompt=prompt, model=model, candidate_count=3, temperature=temperature, max_output_tokens=1000)
  return answer.candidates[0]['output']

In [34]:
temperature = 0.65
query = "How do you shift gears in the Google car?"
answer(text_model, query, db, temperature)

'To shift gears in the Google car, simply move the shift lever to the desired position. Park, Reverse, Neutral, Drive, and Low.'